## Import TensorFlow and other libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
# !tensorboard --bind_all --logdir=my_log_dir

In [ ]:
# Importing the library
import psutil

# Getting % usage of virtual_memory ( 3rd field)
print("RAM memory % used:", psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print("RAM Used (GB):", psutil.virtual_memory()[3] / 1000000000)

In [ ]:
import datetime
import pathlib
import os
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

In [ ]:
from GAF.preprocessing.features_extractors.raw_extractor import RawExtractor
from GAF.preprocessing.data_translator.gaf_translator import GafTranslator
from GAF.preprocessing.preprocessor import Preprocessor

SAMPLE_LENGTH_SECONDS = 2
WINDOWS = 4
BATCH_SIZE = 100
EVALUATION_BATCH_SIZE = 4000
RESAMPLE_MS = SAMPLE_LENGTH_SECONDS * 1000 // WINDOWS
DATAPOINTS_PER_SECOND = 60
DATA_PATH = "/sise/yos-group/royhersh"
PATH = f"{DATA_PATH}/data/processed/fixed_{DATAPOINTS_PER_SECOND}"
ANOMALIES_PATH = f"{DATA_PATH}/anomalies/processed/fixed_{DATAPOINTS_PER_SECOND}"
AXIS_WINDOWS_AMOUNT = (
    SAMPLE_LENGTH_SECONDS * DATAPOINTS_PER_SECOND
)  # 4 windows * 0.500 s/window * 60 points-in-sample/s = 120 points-in-sample
INPUT_SHAPE = (AXIS_WINDOWS_AMOUNT, AXIS_WINDOWS_AMOUNT)
AXES = ["x", "y", "z", "tot"]

extractor = RawExtractor(resample_amount=RESAMPLE_MS)
preprocess = Preprocessor(
    extractor,
    translators=[GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS)],
    packed_windows=WINDOWS,
    path=PATH,
)
anomalies_preprocess = Preprocessor(
    extractor,
    translators=[GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS)],
    packed_windows=WINDOWS,
    path=ANOMALIES_PATH,
)

In [ ]:
OUTPUT_FILE = ""
FILE_TYPE = "h5"


def define_output_path():
    global OUTPUT_FILE
    OUTPUT_FILE = (
        "models/gaf_autoencoders/{extractor}/{time}/".format(
            extractor=str(extractor),
            time=datetime.datetime.now().strftime("%y.%m.%d/%H.%M"),
        )
        + "/{file_name}.{file_type}"
    )
    print(OUTPUT_FILE)


define_output_path()

In [ ]:
from GAF.preprocessing.gaf_dataset_loader import GAFDatasetLoader

sensor = "gyroscope"
translator = GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS, method="summation")
# translator = GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS, method='difference')
train_dataset = GAFDatasetLoader(preprocess.get_file_name(sensor, [translator]), AXES, batch_size=BATCH_SIZE)
anomaly_dataset = GAFDatasetLoader(
    anomalies_preprocess.get_file_name(sensor, [translator]),
    AXES,
    batch_size=BATCH_SIZE,
)

In [ ]:
import json


def load_dataset_shuffle(indicies_path: str):
    with open(indicies_path, "r") as f:
        content = f.read()
        indicies = json.loads(content)
    train_dataset._train_indices = np.array(indicies["benign_train"])
    train_dataset._test_indices = np.array(indicies["benign_test"])

    anomaly_dataset._train_indices = np.array(indicies["anomaly_train"])
    anomaly_dataset._test_indices = np.array(indicies["anomaly_test"])


# load_dataset_shuffle("train_tf/models/gaf_autoencoders/RawExtractor/23.09.11/07.23/train_test_indicies.json")

Plot a normal sample. 

In [ ]:
def simple_plot(title, data):
    data = np.array(data).reshape(INPUT_SHAPE)
    plt.clf()
    plt.figure()
    plt.pcolor(
        np.arange(0, AXIS_WINDOWS_AMOUNT, 1.0),
        np.arange(0, AXIS_WINDOWS_AMOUNT, 1.0),
        data,
        cmap="rainbow",
        vmin=0,
        vmax=1,
    )
    plt.title(title)
    plt.colorbar()
    plt.show()


def simple_subplot(title, data, subplot_data):
    data = np.array(data).reshape(INPUT_SHAPE)
    plt.subplot(*subplot_data)
    plt.pcolor(
        np.arange(0, AXIS_WINDOWS_AMOUNT, 1.0),
        np.arange(0, AXIS_WINDOWS_AMOUNT, 1.0),
        data,
        cmap="rainbow",
        vmin=0,
        vmax=1,
    )
    plt.title(title)
    plt.colorbar()


def plot_groups(title_format, data):
    FIG_SIZE = 4
    fig_width = FIG_SIZE + 2
    for i in range(min(3, len(data))):
        plt.figure(figsize=(len(AXES) * fig_width, FIG_SIZE))
        for axis in range(len(AXES)):
            simple_subplot(
                title_format.format(sensor=sensor, i=i, axis=AXES[axis]),
                data[i][axis],
                (1, 4, axis + 1),
            )
        plt.show()


def draw_data(title_format, dataset):
    dataset._batch_size = 3
    plot_groups(title_format, dataset[0])

In [ ]:
draw_data("Benign {sensor} sample {i} axis {axis}", train_dataset.get_train())

Plot an anomalous samples.

In [ ]:
draw_data("Anomalous {sensor} sample {i} axis {axis}", anomaly_dataset.get_train())

### Build the model

In [ ]:
class AnomalyDetector(Model):
    def __init__(self, encoding_size=300):
        super(AnomalyDetector, self).__init__()
        self.encoding_size = encoding_size
        self.encoder = tf.keras.Sequential(
            [
                layers.InputLayer(
                    input_shape=(*INPUT_SHAPE, 1),
                    batch_size=BATCH_SIZE,
                    dtype=tf.dtypes.float32,
                ),
                layers.Conv2D(filters=1, kernel_size=(5, 5), strides=(2, 2), activation="relu"),
                layers.Flatten(),
                layers.Dense(1800, activation="relu"),
                layers.Dropout(0.2),
                layers.Dense(encoding_size, activation="relu"),
            ]
        )

        self.decoder = tf.keras.Sequential(
            [
                layers.Dense(1800, activation="relu"),
                layers.Dropout(0.2),
                layers.Dense(3600, activation="relu"),  # manual adapt to INPUT_SHAPE
                layers.Reshape((60, 60, 1)),
                layers.Conv2DTranspose(
                    filters=1,
                    kernel_size=(3, 3),
                    strides=(2, 2),
                    activation="relu",
                    padding="same",
                ),
                layers.Reshape(INPUT_SHAPE, dtype=tf.dtypes.float32),
            ]
        )

    def call(self, x):
        # x = tf.expand_dims(x, axis=-1)
        return self.decoder(self.encoder(x))

    def save_model(self, file_name: str, extra_detail: str = ""):
        print(f"Saving model {self} to {OUTPUT_FILE.format(file_name=f'{file_name}*', file_type=FILE_TYPE)}")
        self.encoder.save(
            OUTPUT_FILE.format(file_name=f"{file_name}_encoder{extra_detail}", file_type=FILE_TYPE),
            save_format=FILE_TYPE,
        )
        self.decoder.save(
            OUTPUT_FILE.format(file_name=f"{file_name}_decoder{extra_detail}", file_type=FILE_TYPE),
            save_format=FILE_TYPE,
        )

    def load_model(self, file_name: str, extra_detail: str = ""):
        print(f"Loading model from {OUTPUT_FILE.format(file_name=f'{file_name}*', file_type=FILE_TYPE)}")
        self.encoder = tf.keras.models.load_model(
            OUTPUT_FILE.format(file_name=f"{file_name}_encoder{extra_detail}", file_type=FILE_TYPE)
        )
        self.decoder = tf.keras.models.load_model(
            OUTPUT_FILE.format(file_name=f"{file_name}_decoder{extra_detail}", file_type=FILE_TYPE)
        )

In [ ]:
class AnomalyDetectors:
    def __init__(self, dataset, axes=AXES, encoding_size=300):
        self.new_model = True
        self.encoding_size = encoding_size
        self._dataset = dataset
        self._axes = axes
        self._auto_encoders_amount = len(axes)
        self._start_index = 0
        self._auto_encoders = [AnomalyDetector(encoding_size) for _ in self.get_axis_range()]
        for axis_autoencoder in self._auto_encoders:
            axis_autoencoder.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredError())
            # axis_autoencoder.compile(optimizer='adam', loss=CustomAccuracy(), metrics=['mae', 'mse'])

    def encoder(self, samples):
        output = []
        for index in self.get_axis_range():
            samples_axis = np.asarray([data[index] for data in samples], dtype=np.float32)
            output.append(self._auto_encoders[index].encoder(samples_axis).numpy())
        return list(zip(*output))

    def decoder(self, encoded_samples: np.ndarray) -> np.ndarray:
        output = []
        for index in self.get_axis_range():
            encoded_samples_axis = np.asarray([data[index] for data in encoded_samples], dtype=np.float32)
            output.append(self._auto_encoders[index].decoder(encoded_samples_axis).numpy())
        return list(zip(*output))

    def get_axis_range(self):
        return range(self._start_index, self._auto_encoders_amount)

    def call(self, samples):
        output = []
        for index in self.get_axis_range():
            samples_axis = np.asarray([data[index] for data in samples], dtype=np.float32)
            output.append(self._auto_encoders[index].call(samples_axis).numpy())
        return list(zip(*output))

    def train(self, epochs=50):
        histories = [[] for _ in self.get_axis_range()]
        train_data = self._dataset.get_train()
        test_data = self._dataset.get_test()
        length_train = len(train_data)
        length_test = len(test_data)
        # print(f"Training data {length_train}")
        # print(f"Test data {length_test}")
        for epoch in range(epochs):
            # print(f"Starting epoch {epoch}")
            for batch_index in range(length_train):
                # print(f"train_load {epoch}-{batch_index}/{length_train}")
                current_train = train_data[batch_index]
                batch_loss = []
                for axis in self.get_axis_range():
                    train_cur = np.asarray([data[axis] for data in current_train], dtype=np.float32)
                    assert (
                        len(train_cur) == BATCH_SIZE
                    ), f"[x] No train data on {batch_index}-{axis} - {len(train_cur)}!={BATCH_SIZE} - {np.array(current_train).shape} {train_cur.shape}"
                    loss = self._auto_encoders[axis].train_on_batch(train_cur, train_cur)
                    batch_loss.append(loss)
                    histories[axis].append(loss)
                # print(f"Batch round loss {','.join(map(str, batch_loss))}")
            train_data.on_epoch_end()
        return histories

    def save_model(self, file_name: str):
        for index, axis in enumerate(self._axes):
            self._auto_encoders[index].save_model(file_name, axis)

    def load_model(self, file_name: str):
        self.new_model = False
        for index, axis in enumerate(self._axes):
            self._auto_encoders[index].load_model(file_name, axis)

In [ ]:
def get_model_file_name(autoencoder):
    return f"{sensor}_{extractor._ms_resample}x{preprocess._packed_windows}_{INPUT_SHAPE}_{autoencoder.encoding_size}"


def load_model(autoencoder, model_date: str):
    global OUTPUT_FILE
    OUTPUT_FILE = (
        "models/gaf_autoencoders/{extractor}/{time}".format(extractor=str(extractor), time=model_date)
        + "/{file_name}.{file_type}"
    )
    autoencoder.load_model(get_model_file_name(autoencoder))


def save_model(autoencoder):
    os.makedirs(pathlib.Path(OUTPUT_FILE).parent, exist_ok=True)
    autoencoder.save_model(get_model_file_name(autoencoder))


# load_model(autoencoder, "11.06.23/23.02")

Notice that the autoencoder is trained using only the normal samples, but is evaluated using the full test set.

In [ ]:
def plot_histories(histories):
    for index in range(len(histories)):
        plt.plot(histories[index], label=f"Training Loss axis {AXES[index]}")
        plt.legend()

You will soon classify an samples as anomalous if the reconstruction error is greater than one standard deviation from the normal training examples. First, let's plot a normal samples from the training set, the reconstruction after it's encoded and decoded by the autoencoder, and the reconstruction error.

In [ ]:
# EVALUATION_BATCH_SIZE = BATCH_SIZE
TRAIN_DATASET = train_dataset.get_train()
TRAIN_DATASET._batch_size = EVALUATION_BATCH_SIZE
TEST_DATASET = train_dataset.get_test()
TEST_DATASET._batch_size = EVALUATION_BATCH_SIZE
print(f"Loading train_batch {EVALUATION_BATCH_SIZE}")
train_batch = TRAIN_DATASET[0]
print(f"Loading test_batch {EVALUATION_BATCH_SIZE}")
test_batch = TEST_DATASET[0]

In [ ]:
def plot_decoded_original(title, test_data, decoded_data):
    simple_plot(f"Input {title}", test_data)
    simple_plot(f"Reconstructed {title}", decoded_data)


def plot_original_vs_reconstructed(title_format, reconstructed_title_format, data, reconstructed_data):
    FIG_SIZE = 9
    fig_width = FIG_SIZE - 3
    for i in range(min(3, len(data))):
        plt.figure(figsize=(len(AXES) * fig_width, FIG_SIZE))
        for axis in range(len(AXES)):
            simple_subplot(
                title_format.format(sensor=sensor, i=i, axis=AXES[axis]),
                data[i][axis],
                (2, len(AXES), axis + 1),
            )
        for axis in range(len(AXES)):
            simple_subplot(
                reconstructed_title_format.format(sensor=sensor, i=i, axis=AXES[axis]),
                reconstructed_data[i][axis],
                (2, len(AXES), axis + 1 + len(AXES)),
            )
        plt.show()

In [ ]:
def encode_decode_plot(autoencoder, data_batch, title_format, reconstructed_title_format):
    print(f"Encode {len(data_batch)}")
    encoded_data = autoencoder.encoder(data_batch)
    print(f"Decode {len(encoded_data)}")
    decoded_data = autoencoder.decoder(encoded_data)
    print(
        np.array(test_batch).shape,
        np.array(encoded_data).shape,
        np.array(decoded_data).shape,
    )
    plot_original_vs_reconstructed(title_format, reconstructed_title_format, test_batch, decoded_data)

Create a similar plot, this time for an anomalous test example.

In [ ]:
ANOMALY_TRAIN_DATASET = anomaly_dataset.get_train()
ANOMALY_TRAIN_DATASET._batch_size = EVALUATION_BATCH_SIZE
ANOMALY_TEST_DATASET = anomaly_dataset.get_test()
ANOMALY_TEST_DATASET._batch_size = EVALUATION_BATCH_SIZE
print(f"Loading anomaly_train_batch {EVALUATION_BATCH_SIZE}")
anomaly_train_batch = ANOMALY_TRAIN_DATASET[0]
print(f"Loading anomaly_test_batch {EVALUATION_BATCH_SIZE}")
anomaly_test_batch = ANOMALY_TEST_DATASET[0]

### Detect anomalies

Detect anomalies by calculating whether the reconstruction loss is greater than a fixed threshold. In this tutorial, you will calculate the mean average error for normal examples from the training set, then classify future examples as anomalous if the reconstruction error is higher than one standard deviation from the training set.


Plot the reconstruction error on normal samples from the training set

In [ ]:
LOSS_ARRANGE_SIZE = AXIS_WINDOWS_AMOUNT * AXIS_WINDOWS_AMOUNT


def calculate_loss(model, data):
    print(data.shape)
    reconstructions = model.call(data)
    print(np.array(reconstructions).shape)
    # custom_acc = CustomAccuracy()
    custom_acc = tf.keras.losses.MeanSquaredError()
    amount, axes, _data1, _data2 = data.shape
    loss = []
    for i in range(amount):
        tmp_loss = 0
        for axis in model.get_axis_range():
            tmp_loss += custom_acc.call(
                np.array(reconstructions[i][axis - model._start_index]).reshape(LOSS_ARRANGE_SIZE),
                np.array(data[i][axis]).reshape(LOSS_ARRANGE_SIZE),
            )
        loss.append(tmp_loss)
    return loss

In [ ]:
np.array(train_batch).shape

In [ ]:
def calculate_loss_span(autoencoder, data_batch, title=""):
    train_loss = calculate_loss(autoencoder, data_batch)
    loss_mean = np.mean(train_loss)
    loss_std = np.std(train_loss)
    plt.hist(train_loss, bins=50)
    plt.title(title)
    plt.xlabel("Loss")
    plt.ylabel("No of examples")
    plt.show()
    return loss_mean, loss_std

Choose a threshold value that is one standard deviations above the mean.

In [ ]:
threshold = -1
anomaly_threshold = -1
anomaly_benign_test_span_distance = -1
benign_train_span = ""
benign_test_span = ""
anomaly_train_span = ""


def calculate_thresholds(autoencoder):
    global threshold, anomaly_threshold, benign_train_span, benign_test_span, anomaly_train_span
    tmp_mean, tmp_std = calculate_loss_span(autoencoder, train_batch, "Benign train loss")
    benign_train_span = f"[{tmp_mean - tmp_std}-{tmp_mean + tmp_std}]"
    print("Train loss span: ", benign_train_span)
    threshold = tmp_mean + tmp_std
    print("Benign train threshold: ", threshold)

    tmp_mean, tmp_std = calculate_loss_span(autoencoder, test_batch, "Benign test loss")
    benign_test_upper_loss_span = tmp_mean + tmp_std
    benign_test_span = f"[{tmp_mean - tmp_std}-{benign_test_upper_loss_span}]"
    print("Benign test loss span: ", benign_test_span)

    tmp_mean, tmp_std = calculate_loss_span(autoencoder, anomaly_train_batch, "Anomaly train loss")
    anomaly_train_span = f"[{tmp_mean - tmp_std}-{tmp_mean + tmp_std}]"
    print(
        "benign train-test, anomalous-train span: ",
        benign_train_span,
        benign_test_span,
        anomaly_train_span,
    )
    anomaly_lower_span = tmp_mean - tmp_std
    anomaly_threshold = anomaly_lower_span
    anomaly_benign_test_span_distance = anomaly_lower_span - benign_test_upper_loss_span
    print("Anomaly train threshold: ", anomaly_lower_span)

Note: There are other strategies you could use to select a threshold value above which test examples should be classified as anomalous, the correct approach will depend on your dataset. You can learn more with the links at the end of this tutorial. 

If you examine the reconstruction error for the anomalous examples in the test set, you'll notice most have greater reconstruction error than the threshold. By varing the threshold, you can adjust the [precision](https://developers.google.com/machine-learning/glossary#precision) and [recall](https://developers.google.com/machine-learning/glossary#recall) of your classifier. 

Classify an samples as an anomaly if the reconstruction error is greater than the threshold.

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)


def predict(model, data, threshold):
    loss = calculate_loss(model, data)
    return tf.math.less(loss, threshold)


# cannot memoize without threshold with the following memoize function
def get_stats(predictions, labels):
    return f"Accuracy = {accuracy_score(labels, predictions)}"


def get_stats_all(predictions, labels):
    return f"""{get_stats(predictions, labels)}
Precision = {precision_score(labels, predictions)}
Recall = {recall_score(labels, predictions)}
F1 = {f1_score(labels, predictions)}
confusion_matrix -\n{confusion_matrix(labels, predictions)}
"""

In [ ]:
from itertools import repeat, chain


def evaluate(model, threshold):
    output = f"\n[*] Calculating result with threshold {threshold}"
    predications_benign = predict(model, test_batch, threshold)
    labels_benign = list(repeat(True, len(predications_benign)))
    stats_benign = get_stats(predications_benign, labels_benign)
    output += "\n~ Benign test performance ~\n" + stats_benign

    predications_anomaly = predict(model, anomaly_train_batch, threshold)
    labels_anomaly = list(repeat(False, len(predications_anomaly)))
    stats_anomaly = get_stats(predications_anomaly, labels_anomaly)
    output += "\n~ Anomaly train performance ~\n" + stats_anomaly

    predications_anomaly_test = predict(model, anomaly_test_batch, threshold)
    labels_anomaly_test = list(repeat(False, len(predications_anomaly_test)))
    stats_anomaly = get_stats(predications_anomaly_test, labels_anomaly_test)
    output += "\n~ Anomaly test performance ~\n" + stats_anomaly

    predications_total = list(chain(predications_benign, predications_anomaly, predications_anomaly_test))
    labels_total = list(chain(labels_benign, labels_anomaly, labels_anomaly_test))
    stats_total = get_stats_all(predications_total, labels_total)
    output += "\nTotal performance: \n" + stats_total
    return output

In [ ]:
def log_results(autoencoder, file_name: str):
    output_filename = OUTPUT_FILE.format(file_name=file_name, file_type="log")
    with open(output_filename, "w+") as f:
        date = datetime.datetime.now().strftime("%d/%m/%y %H:%M")
        f.write("\n\n~~~ Model Results ~~~\n")
        f.write(f"Finish date: {date}\n")
        f.write(f"Output directory: {output_filename}\n")
        f.write(f"AXES: {AXES}\n")
        f.write(f"Sample length seconds: {SAMPLE_LENGTH_SECONDS}\n")
        f.write(f"Enforced datepoints per second: {DATAPOINTS_PER_SECOND}\n")
        f.write(f"Resample: {RESAMPLE_MS}\n")
        f.write(f"INPUT_SHAPE: {INPUT_SHAPE}\n")

        f.write("\n\n!! Model Info !!\n")
        sub_autoencoder = autoencoder._auto_encoders[0]
        f.write(f"ENCODING_SIZE: {sub_autoencoder.encoding_size}\n")
        f.write(f"Encoder:\n")
        sub_autoencoder.encoder.summary(print_fn=lambda x: f.write(x + "\n"))
        f.write(f"Decoder:\n")
        sub_autoencoder.decoder.summary(print_fn=lambda x: f.write(x + "\n"))

        f.write("\n\n!! Dataset Info !!\n")
        f.write(f"Sensor: {sensor}\n")
        f.write(f"Train: <data: {len(train_dataset.get_train())}>, Test:<data: {len(train_dataset.get_test())}>\n")
        f.write(
            f"Anomalies: <data: {len(anomaly_dataset.get_train())}>, Test:<data: {len(anomaly_dataset.get_test())}>\n"
        )
        f.write(f"Benign train span: {benign_train_span}\n")
        f.write(f"Benign test span: {benign_test_span}\n")
        f.write(f"Anomaly train span: {anomaly_train_span}\n")
        f.write(f"Anomaly-train:Benign-test span distance: {anomaly_benign_test_span_distance}\n")

        f.write("\n\n!! First evaluation !!\n")
        f.write(evaluate(autoencoder, threshold))
        f.write("\n\n!! Second evaluation !!\n")
        f.write(evaluate(autoencoder, anomaly_threshold))

In [ ]:
def set_axes(autoencoder, axes, start_index, end_index):
    autoencoder._axes = axes
    autoencoder._auto_encoders_amount = end_index
    autoencoder._start_index = start_index
    set_axes_dataset(axes)


def set_axes_dataset(axes):
    global train_dataset, anomaly_dataset
    train_dataset._axes = axes
    anomaly_dataset._axes = axes

In [ ]:
def train_model_and_save_results(axes, encoding_size):
    autoencoder = AnomalyDetectors(train_dataset, axes, encoding_size)
    histories = autoencoder.train(epochs=30)
    plot_histories(histories)
    encode_decode_plot(
        autoencoder,
        test_batch[0:3],
        "Input {sensor} sample {i} axis {axis}",
        "Reconstructed {sensor} sample {i} axis {axis}",
    )
    encode_decode_plot(
        autoencoder,
        anomaly_train_batch[0:3],
        "Input {sensor} anomalous sample {i} axis {axis}",
        "Reconstructed {sensor} anomalous sample {i} axis {axis}",
    )
    calculate_thresholds(autoencoder)
    # print(evaluate(autoencoder, anomaly_threshold))
    if autoencoder.new_model:
        save_model(autoencoder)
        log_results(autoencoder, f"{'_'.join(AXES)}_{get_model_file_name(autoencoder)}")
    return autoencoder

In [ ]:
def produce_model_and_evaluate_axes(encoding_size):
    global AXES
    AXES = ["x", "y", "z", "tot"]
    set_axes_dataset(AXES)
    autoencoder = train_model_and_save_results(AXES, encoding_size)

    AXES = ["x", "y", "z"]
    set_axes(autoencoder, AXES, 0, len(AXES))
    calculate_thresholds(autoencoder)
    log_results(autoencoder, f"{'_'.join(AXES)}_{get_model_file_name(autoencoder)}")

    AXES = ["tot"]
    set_axes(autoencoder, AXES, 3, 4)
    calculate_thresholds(autoencoder)
    log_results(autoencoder, f"{'_'.join(AXES)}_{get_model_file_name(autoencoder)}")

In [ ]:
def run_for_encoding_size():
    for encoding_size in ENCODING_SIZES:
        print(datetime.datetime.now().strftime(f"{encoding_size}-started at: %d.%m.%y-%H:%M"))
        define_output_path()
        produce_model_and_evaluate_axes(encoding_size)
        print(datetime.datetime.now().strftime(f"{encoding_size}-ended at: %d.%m.%y-%H:%M"))

In [ ]:
import json


def save_train_test_idicies(file_name: str = "train_test_indicies"):
    out = {
        "benign_train": train_dataset._train_indices.tolist(),
        "benign_test": train_dataset._test_indices.tolist(),
        "anomaly_train": anomaly_dataset._train_indices.tolist(),
        "anomaly_test": anomaly_dataset._test_indices.tolist(),
    }
    os.makedirs(pathlib.Path(OUTPUT_FILE).parent, exist_ok=True)
    with open(OUTPUT_FILE.format(file_name=file_name, file_type="json"), "w+") as f:
        f.write(json.dumps(out))

In [ ]:
save_train_test_idicies(f"train_test_indicies_{DATAPOINTS_PER_SECOND}")
ENCODING_SIZES = [250, 300, 400, 500, 600, 900, 1200]
run_for_encoding_size()

In [ ]:
# wait for autosave
import time

time.sleep(180)

In [ ]:
job_cancel_str = "scancel " + os.environ["SLURM_JOBID"]
os.system(job_cancel_str)